<h1>Детектирование объектов на изображениях</h1>

<h3>Обучение и тестирование модели детектора</h3>

Источники:
- https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/README.md
- https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/index.html
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

Вы подготовили размеченный набор данных, состоящий из обучающего набора изображений, тестового набора изображений и набора изображений для оценки работы детектора.

Сейчас вам необходимо войти в свой облачный диск на Google и в его корневом каталоге создать папку проекта (например, **neural_network**).

После этого загрузите в каталог проекта папку **images** с набором данных для обучения

Подключитесь к облачному диску Google и перейдите в свою папку проекта

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

WORK_DIR = '/content/gdrive/My Drive/neural_network'

%cd '{WORK_DIR}'

Внимательно читайте следующие инструкции и выполняйте указанные команды. Удачи!

# Подготовка программного окружения

* Загрузите библиотеки моделей TensorFlow

In [ ]:
!git clone https://github.com/tensorflow/models.git

* Установите все связанные библиотеки и инструменты

In [ ]:
%cd '{WORK_DIR}/models/research/'
!pip install protobuf==3.20.3
!protoc object_detection/protos/*.proto --python_out=.

* Скопируйте файл установки TF Object Detection API и установите библиотеку (установка будет происходить в скрытом режиме)

In [ ]:
%%capture
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .



*   Понизьте версию TensorFlow для стабильности



In [ ]:
%%capture
!pip install tensorflow==2.13.1

* Добавьте системную переменную окружения path путь к каталогу research скачанного репозитория моделей TensorFlow

In [ ]:
import sys
sys.path.append('{WORK_DIR}/models/research')



*   Установите CUDA



In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!apt-get update && sudo apt-get install cuda-toolkit-11-0
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

* Протестируйте установленные библиотеки

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

WORK_DIR = '/content/gdrive/My Drive/neural_network'
%cd '{WORK_DIR}/models/research/'

!pip install protobuf==3.20.3
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# Создание карты меток и генерирование файлов TFRecord

* Создайте папку annotations в каталоге своего проекта

In [ ]:
import os
%cd '{WORK_DIR}'
if not(os.path.exists('annotations')):
    os.mkdir('annotations')

* Создайте список словарей, каждый из которых хранит название метки (name) и её номер (id), а затем запишите его в файл в папке annotations с названием label_map.pbtxt, который будет хранить данные о метках

In [ ]:
labels = [{'name':'ironman', 'id':1},
          {'name':'mandalorian', 'id':2},
          {'name':'thanos', 'id':3}]

with open('annotations/label_map.pbtxt', 'w', encoding="utf-8") as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

* Создайте папку scripts в каталоге проекта и загрузите в неё скрипт generate_tfrecord.py

In [ ]:
import os
%cd '{WORK_DIR}'
if not(os.path.exists('scripts')):
    os.mkdir('scripts')
%cd '{WORK_DIR}/scripts'
!wget "https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py"

* Запустить скрипт для генерирования файлов TFRecord набора обучающих данных и набора тестовых данных

In [ ]:
%cd '{WORK_DIR}'
!python scripts/generate_tfrecord.py -x 'images/train' -l 'annotations/label_map.pbtxt' -o 'annotations/train.record'
!python scripts/generate_tfrecord.py -x 'images/test' -l 'annotations/label_map.pbtxt' -o 'annotations/test.record'

# Подготовка и трансферное обучение модели детектора

* создайте папку my_models, загрузите в неё обученную модель из коллекции [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) и скопируйте файл pipeline в каталог своей модели для дальнейшей переконфигурации

In [ ]:
%cd '{WORK_DIR}'

MODEL_DIR_DOWNLOAD = 'model_downloaded'
MODEL_DIR_MY = 'model_my'

os.makedirs(MODEL_DIR_DOWNLOAD, exist_ok=True)

!wget -P {MODEL_DIR_DOWNLOAD} "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
!tar -zxvf {MODEL_DIR_DOWNLOAD}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz -C {MODEL_DIR_DOWNLOAD}

os.makedirs(MODEL_DIR_MY, exist_ok=True)

!cp {MODEL_DIR_DOWNLOAD}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config {MODEL_DIR_MY}/

* скопируйте файлы для обучения и экспорта модели детектора в корневую папку проекта

In [ ]:
%cd '{WORK_DIR}'
!cp models/research/object_detection/model_main_tf2.py .
!cp models/research/object_detection/exporter_main_v2.py .
!cp models/research/object_detection/export_tflite_graph_tf2.py .

* Обновите конфигурацию для трансферного обучения в файле pipeline.config, указав количество классов, размер пакета, файл чекпойнта обученной модели, файлы карты меток и TFRecord

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

PIPELINE_FILE = f'{MODEL_DIR_MY}/pipeline.config'

config = config_util.get_configs_from_pipeline_file(f'{PIPELINE_FILE}')

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(f'{PIPELINE_FILE}', "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.num_steps = 2000
pipeline_config.train_config.fine_tune_checkpoint = f'{MODEL_DIR_DOWNLOAD}/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= 'annotations/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = ['annotations/train.record']
pipeline_config.eval_input_reader[0].label_map_path = 'annotations/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ['annotations/test.record']

config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(f'{PIPELINE_FILE}', "wb") as f:
    f.write(config_text)

* Запустите обучение модели детектора с выбранным количеством шагов

In [ ]:
!python model_main_tf2.py \
    --pipeline_config_path={PIPELINE_FILE} \
    --model_dir={MODEL_DIR_MY} \
    --alsologtostderr \
    --sample_1_of_n_eval_examples=1

* Извлеките обученную модель детектора для дальнейшего использования в алгоритмах распознавания объектов

In [ ]:
!python exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path={PIPELINE_FILE} \
    --trained_checkpoint_dir={MODEL_DIR_MY} \
    --output_directory={MODEL_DIR_MY}/exported

In [ ]:
!python export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {MODEL_DIR_MY} \
    --output_directory {MODEL_DIR_MY}/tflite \
    --pipeline_config_path {PIPELINE_FILE}

In [ ]:
# Convert exported graph file into TFLite model file
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(f'{MODEL_DIR_MY}/tflite/saved_model')
tflite_model = converter.convert()

with open(f'{MODEL_DIR_MY}/tflite/model.tflite', 'wb') as f:
  f.write(tflite_model)

# Тестирование детектирования объектов на изображениях

* Загрузите сохранённую обученную модель

In [ ]:
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL=f"{MODEL_DIR_MY}/exported/saved_model"

print('Loading model...', end='')
# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

* Загрузите карту меток

In [ ]:
category_index=label_map_util.create_category_index_from_labelmap(f"{WORK_DIR}/annotations/label_map.pbtxt",use_display_name=True)

* Загрузите список изображений из папки images/eval

In [ ]:
from imutils import paths
import random

images_test = list(paths.list_images(f"{WORK_DIR}/images/test"))

random.shuffle(images_test)
print(images_test)

* Запустите алгоритм детектирования объектов на подготовленных изображениях и наблюдайте за результатом

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import math

def load_image_into_numpy_array(path):
  return np.array(Image.open(path))

%matplotlib inline
num_images = len(images_test)
cols = 3  # Количество изображений в ряду
rows = math.ceil(num_images / cols)  # Округляем вверх, чтобы учесть все изображения

plt.figure(figsize=(24, rows * 5))  # Автоматический размер


for i, image_path in enumerate(images_test):
  plt.subplot(rows, cols, i + 1)

  image_np = load_image_into_numpy_array(image_path)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  detections = detect_fn(input_tensor)
  num_detections = int(detections.pop('num_detections'))
  detections = {key:value[0,:num_detections].numpy() for key,value in detections.items()}
  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections=image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=100,
        min_score_thresh=.5,
        agnostic_mode=False)

  plt.imshow(image_np_with_detections)
  plt.axis('off')